<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/AI-sustainability-agent/blob/main/Getting_data_from_iOT_sensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collecting data from a variety of IoT sensors generally involves a multi-step process that includes data gathering, data preprocessing, and data storage. Below is a Python code example that demonstrates a simplified version of these steps. This example assumes that each sensor has a Python function to read its data, and we're using MQTT (a lightweight messaging protocol for IoT) to simulate the data collection from IoT devices.

In [1]:
!pip install paho-mqtt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62118 sha256=95f9cc08e2fef9045195b2db15ca0d1087d9c43df0333c5c6cd4b0352476fd23
  Stored in directory: /root/.cache/pip/wheels/8b/bb/0c/79444d1dee20324d442856979b5b519b48828b0bd3d05df84a
Successfully built paho-mqtt


The read_*_sensor functions simulate the reading of data from various IoT sensors.

The collect_sensor_data function collects data from all sensors and updates the SENSOR_DATA dictionary.

The MQTT-related code initializes an MQTT client, connects to a broker, and subscribes to a topic.

The main loop (if __name__ == "__main__":) collects sensor data, converts it to a JSON-formatted string, and publishes it to an MQTT topic.

In [ ]:
import json
import random
import time
from paho.mqtt import client as mqtt_client

# Initialize 'Database' to store sensor data
SENSOR_DATA = {
    'temperature': 0,  # degrees Celsius
    'humidity': 0,  # percentage
    'energy_consumption': 0,  # kWh
    'water_flow': 0  # Liters per minute
}

# Functions to read data from sensors (Simulated)
def read_temperature_sensor():
    return random.uniform(20, 30)

def read_humidity_sensor():
    return random.uniform(40, 60)

def read_energy_sensor():
    return random.uniform(1000, 2000)

def read_water_flow_sensor():
    return random.uniform(10, 20)

# Function to collect data from all sensors
def collect_sensor_data():
    SENSOR_DATA['temperature'] = read_temperature_sensor()
    SENSOR_DATA['humidity'] = read_humidity_sensor()
    SENSOR_DATA['energy_consumption'] = read_energy_sensor()
    SENSOR_DATA['water_flow'] = read_water_flow_sensor()

# MQTT settings
broker = 'mqtt.eclipse.org'
port = 1883
topic = "sustainability/sensor_data"

# MQTT callback for when a message is received
def on_message(client, userdata, message):
    print(f"Received message '{message.payload.decode()}' on topic '{message.topic}'")

# Initialize MQTT client
client = mqtt_client.Client()
client.on_message = on_message
client.connect(broker, port)
client.subscribe(topic)

# Main loop to collect and send sensor data
if __name__ == "__main__":
    client.loop_start()

    while True:
        collect_sensor_data()
        payload = json.dumps(SENSOR_DATA)

        # Publish sensor data to MQTT topic
        client.publish(topic, payload)
        print(f"Published sensor data: {payload}")

        time.sleep(5)  # Collect data every 5 seconds

    client.loop_stop()


HOW DO WE GATHER DATA FROM IoT devices

Why Understanding the API is Critical

IoT sensors can vary in complexity and functionality, and the means by which you interact with these sensors is usually through their Application Programming Interface, or API. An API is essentially a set of rules and protocols that allow one piece of software or hardware to interact with another. These rules can include the URL you need to connect to, the data format (usually JSON or XML), and any required headers or parameters. Failure to understand these elements will lead to unsuccessful data retrieval or modification attempts. Essentially, knowing how the sensor's API works is akin to knowing how to speak its "language."

----------
Base URL: The root URL under which all API endpoints reside.

Endpoints: Specific URLs for various functionalities (e.g., getting temperature data, updating sensor settings, etc.).

Request Methods: Whether you need to use HTTP GET, POST, PUT, or DELETE methods to retrieve or modify data.

Request Headers: Any required HTTP headers, like Authorization for including an API key or authentication token.

Request Parameters: Any data that needs to be passed as part of the URL or in the body of a POST request.




Make a basic API call:

In [ ]:
import requests

def get_sensor_data(api_url):
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None


Gather Data from Different Types of Sensors
Let's say we have three types of sensors: a temperature sensor, a humidity sensor, and an energy consumption sensor. Assume they have RESTful APIs accessible at the following URLs:

Temperature Sensor: http://api.sensors.io/temperature

Humidity Sensor: http://api.sensors.io/humidity

Energy Sensor: http://api.sensors.io/energy

In [ ]:
def collect_all_sensor_data():
    sensors = {
        'temperature': 'http://api.sensors.io/temperature',
        'humidity': 'http://api.sensors.io/humidity',
        'energy': 'http://api.sensors.io/energy'
    }
    collected_data = {}

    for sensor_type, api_url in sensors.items():
        data = get_sensor_data(api_url)
        if data is not None:
            collected_data[sensor_type] = data

    return collected_data


API response parsing

In [ ]:
def get_sensor_data(api_url):
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        json_data = response.json()

        if 'data' in json_data:
            return json_data['data']
        else:
            return None

    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None


Putting it all together:

Add authentication to the API calls.

Handle various types of errors and exceptions more robustly.

Store the collected data into a database.

Add more features like data caching, logging, etc.

In [ ]:
if __name__ == "__main__":
    collected_data = collect_all_sensor_data()
    print("Collected Sensor Data:")
    print(collected_data)
